# THIS SHOULD BE TURNED INTO FUNCTION <br>
## TO LOAD IN CHARTEVENTS, NOTEEVENTS, (labevents, inputevents_cv, inputevents_mv)

In [1]:
import pandas as pd

In [2]:
df_patient_admit_icu = pd.read_pickle('./data/pickle/preproc/df_patient_admit_icu__20210119.pkl')

In [5]:
df_ids = df_patient_admit_icu[['SUBJECT_ID','HADM_ID','HAS_CHARTEVENTS_DATA','INTIME','DAYS_ADM_TO_ICU']]

In [6]:
# import patient info
data_dir = 'data/physionet.org/files/mimiciii/1.4/'
notes_file = 'NOTEEVENTS.csv'

In [ ]:
skiplist = []

row_count = 2083180
skiprows = 0
nrows = 100000  # defualt
usecols = ['ROW_ID','SUBJECT_ID','HADM_ID','CHARTTIME','CATEGORY','DESCRIPTION','ISERROR','TEXT']
cnt=0

while skiprows<=row_count:
    print('Iteration {}...'.format(cnt))
    
    if skiprows + nrows > row_count:
            nrows = row_count - skiprows
        else:
            nrows = 100000
            
    df = pd.read_csv(data_dir + notes_file,sep=',', skiprows=skiprows, nrows=nrows, usecols=usecols)
    # convert charttime to datetime
    df.CHARTTIME = pd.to_datetime(df.CHARTTIME,format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    
    # add to skiplist rows of df that don't have matching ['SUBJECT_ID', 'HADM_ID'] in df_ids
    
    
    # merge
    df = df.merge(df_ids,how='inner',left_on=['SUBJECT_ID', 'HADM_ID'],right_on=['SUBJECT_ID', 'HADM_ID'])
    # calculate days from note event to icu admission
    df['DAYS_NOTE_TO_ICU'] = (df['INTIME'] - df['CHARTTIME']).dt.total_seconds()/(24*60*60)
    
    # add to skiplist rows of df where df['DAYS_NOTE_TO_ICU']<0
    
    # anything else?